In [26]:
import pandas as pd
import re
import numpy as np
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
pd.set_option('display.max_rows', 15)

In [27]:
# read file
df = pd.read_csv('/content/drive/MyDrive/데이터톤2024/bookInformation.txt', encoding='euc-kr')

'''
용도 : 출판년도를 정제하는 function
return : 정제된 출판년도
'''
def clean_publication_year(row):
    year = row['출판년도']  # 출판년도
    book_ID = row['도서ID']
    author = row['저자']
    book_name = row['서명']
    registration_year = row['등록일자']  # 등록일자 (연도만 추출된 상태)

    #case 1: Nul인 경우
    if pd.isnull(year):  # 결측치 처리
        similar_books = df[(df['서명'] == book_name) & (df['저자'] == author)]
        if not similar_books.empty:
            if not similar_books['출판년도'].isnull().all():
                return similar_books['출판년도'].dropna().iloc[0]
            if '등록일자' in similar_books.columns and not similar_books['등록일자'].isnull().all():
                return similar_books['등록일자'].mean()
        return registration_year
    # Case 2: 괄호 없이 4자리 숫자인 경우
    if re.match(r'^\d{4}$', year):  # 4자리 숫자인 경우
        return int(year)

    #case 3: 출판년도에 괄호가 존재
    match_inside = re.search(r'\((\d{4})\)', year)
    match_outside = re.search(r'\b(\d{4})\b', year)
    #case 3-1: 괄호 안과 밖에 숫자가 모두 존재
    if match_inside and match_outside:
        year_inside = int(match_inside.group(1))
        year_outside = int(match_outside.group(1))
        #괄호 안 숫자가 비정상적일 때
        if not (1800 <= year_inside <= 2025):
            return year_outside
        #괄호 안 숫자가 정상적일 때
        return year_inside
    #case 3-2: 괄호 밖에만 숫자가 존재
    if match_outside:
        return int(match_outside.group(1))
    #case 3-3: 괄호 안에만 숫자가 존재
    if match_inside:
        year_inside = int(match_inside.group(1))
        if 1800 <= year_inside <= 2025:
            return year_inside
        else:
            return int(row['등록일자'])

    #case 4 : 2004-2007과 같이 범위가 존재
    if '-' in year:
        match = re.match(r'(\d{4})-\d{4}', year)
        if match:
            return int(match.group(1))  # 가장 빠른 연도 반환
    #case 5: 자세한 년도가 존재하지 않을 때
    if re.match(r'18--', year):
        return 1870
    if re.match(r'19-+|19--|197-|198-|199-|199  -|19 -|19uu|199 -|198', year):  # '19-' 또는 '19--' 처리
        if not pd.isnull(registration_year) and str(registration_year)[:2] == '19':  # 등록일자와 앞 두 자리 비교
            return int(registration_year)
        if re.match(r'19-+|19--|197-|19uu', year):
            return 1975
        if re.match(r'198-|198', year):
            return 1985
        if re.match(r'199-|199  -|19 -|199 -', year):
            return 1995
        return int(registration_year)
    if re.match(r'200-+|20--|200', year):  # '200-' 또는 '20--' 처리
        if not pd.isnull(registration_year) and str(registration_year)[:2] == '20':  # 등록일자와 앞 두 자리 비교
            return int(registration_year)
        return 2005
    if re.match(r'201', year):
        return 2015

    # 괄호 제거
    year = re.sub(r'\(.*?\)', '', year)
    # "c2005"와 같은 경우 앞의 숫자만 추출
    year = re.sub(r'[^\d]', '', year)
    # "200u"와 같은 경우 2007로 대체
    if re.match(r'\d{4}', year):
        return int(year)

    #print(f"book id : {book_ID} Current Year: {year}, Registration Year: {registration_year}")
    #print(f"Unknown format for year: {year}. Using registration year: {registration_year}")
    return registration_year  # 처리할 수 없는 경우 NaN 반환

'''
용도 : '193,24', '291..24'등 format에 맞지 않는 분류 코드를 정제
return : (3자리 수.1~2자리 수) 반환
'''
def formatting_classification_code(code):
    code = str(code)
    code = code.replace(',', '.')
    code = code.replace(' ', '.')
    code = code.replace('C', '.')
    code = re.sub(r'[^\d.]', '', code)
    if code.count('.') > 1:
        code = re.sub(r'\.(?=.*\.)', '', code)
    if '.' in code:
        integer_part, decimal_part = code.split('.', 1)
        integer_part = integer_part.zfill(3)[:3]  # 3자리 정수로 앞에 0 채우기
        decimal_part = decimal_part[:2]  # 소수점 이하 2자리만 유지
        return f"{integer_part}.{decimal_part}"
    else:
        return f"{int(code):03d}"
    return f"{int(code):03d}"

'''
용도 : 분류 코드를 정제함
return : ValueError일 경우 - formatting_classification_code호출 후 정제, 올바른 value일 경우 - 3자리의 정수 반환
'''
def clean_classification_code(code):
    try:
        code = float(code)
        code = int(code // 10 * 10)
        return code
    except(ValueError, TypeError):
        code = formatting_classification_code(code)
        code = float(code)
        code = int(code // 10 * 10)
        return code

'''
용도 : 저자 정보를 정제함
return : null일 경우 - 저자 미상, null이 아닐 경우 - 저자
'''
def clean_author(name):
    if pd.isnull(name):
        return "저자미상"
    return name

#function 호출부
df['등록일자'] = pd.to_datetime(df['등록일자']).dt.year
df['출판년도'] = df.apply(clean_publication_year, axis=1)
df['저자'] = df['저자'].apply(clean_author)
df['분류코드'] = df['분류코드'].apply(clean_classification_code)
df = df.drop('출판사', axis=1)
df = df.drop('ISBN', axis=1)

# 결과 저장
output_path = '/content/drive/MyDrive/데이터톤2024/processed_bookInformation.csv'
df.to_csv(output_path, index=False, encoding='utf-8-sig')  # 결과를 CSV로 저장


             도서ID  등록일자    수서방법  분류코드  \
0       SS_270312  2017    학과신청   190   
1       SS_233793  2008  수서정보없음   160   
2       SS_233788  2008  수서정보없음   160   
3       SS_075978  2010   이용자희망   160   
4       SS_297286  2011    사서선정   160   
...           ...   ...     ...   ...   
301146  SS_156440  2024   이용자희망   900   
301147  SS_156358  2024   이용자희망   150   
301148  SS_156530  2024   이용자희망   790   
301149  SS_156441  2024   이용자희망   150   
301150  SS_156538  2024   이용자희망   930   

                                                       서명  \
0       Le courage de la verite :le gouvernement de so...   
1                                              김나지움 논리학입문   
2                                              김나지움 논리학입문   
3                                               선제논리를 향하여   
4                                               선제논리를 향하여   
...                                                   ...   
301146                                     하루 한 꼭지 초등 세계사   
301147             